# Chuẩn bị dataset

In [1]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle

In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d vlbthambawita/visemtracking

 99% 1.76G/1.78G [00:15<00:00, 136MB/s]
100% 1.78G/1.78G [00:15<00:00, 120MB/s]


In [8]:
!unzip visemtracking.zip

Streaming output truncated to the last 5000 lines.
  inflating: VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_557_with_ftid.txt  
  inflating: VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_558_with_ftid.txt  
  inflating: VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_559_with_ftid.txt  
  inflating: VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_55_with_ftid.txt  
  inflating: VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_560_with_ftid.txt  
  inflating: VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_561_with_ftid.txt  
  inflating: VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_562_with_ftid.txt  
  inflating: VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_563_with_ftid.txt  
  inflating: VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_564_with_ftid.txt  
  inflating: VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_565_with_ftid.txt  
  inflating: VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_566_with_ftid

# Show image

In [9]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import glob

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from torchvision.utils import draw_bounding_boxes
from torchvision.io import read_image
import torchvision
import torch

%matplotlib inline
plt.rcParams["figure.figsize"] = (20,20)

In [10]:
MAIN_DATA_PATH = '/content/VISEM_Tracking_Train_v4/Train'

In [11]:
video_ids = os.listdir(MAIN_DATA_PATH)
print(video_ids)

['29', '15', '13', '19', '14', '11', '82', '22', '21', '23', '12', '38', '54', '52', '60', '36', '30', '35', '47', '24']


In [12]:
# Create widgets
directory = widgets.Dropdown(options=video_ids)
images = widgets.Dropdown(options=os.listdir(MAIN_DATA_PATH + "/" + str(directory.value) + "/images/"))

In [13]:
# Updates the image options based on directory value
def update_images(*args):
    images.options = os.listdir(MAIN_DATA_PATH + "/" + str(directory.value) + "/images/")

In [14]:
# Tie the image options to directory value
directory.observe(update_images, 'value')

In [15]:
def ccwh2xyxy(img_h, img_w, bb_box):
    norm_center_x = bb_box[0]
    norm_center_y = bb_box[1]
    norm_label_width = bb_box[2]
    norm_label_height = bb_box[3]
    
    center_x = norm_center_x * img_w
    center_y = norm_center_y * img_h
    label_width = norm_label_width * img_w
    label_height = norm_label_height * img_h
    
    x_min = center_x - (label_width/2)
    y_min = center_y - (label_height/2)
    x_max = center_x + (label_width/2)
    y_max = center_y + (label_height/2)
    
    return [x_min, y_min, x_max, y_max]

In [16]:
def get_bboex(bb_txt_path, img_h, img_w):
    
    bboxes = []
    classes = []
    with open(bb_txt_path, "r") as f:
        bbs = f.read().splitlines()
    
        for bb in bbs:
            crd = list(map(float, (bb.split())))
            cls_id  = int(crd[0])
            crd = ccwh2xyxy(img_h, img_w, crd[1:]) 
            bboxes.append(crd)
            classes.append(cls_id)
    
    return classes, bboxes

In [17]:
# Show the images
def show_images(fdir, file):
    img_path = MAIN_DATA_PATH + "/" + fdir + "/images/" + file
    bb_txt_path = MAIN_DATA_PATH + "/"+ fdir + "/labels/" + file.split(".")[0] + ".txt"
    #print(bb_txt_path)
    img = read_image(img_path)
    
    img_height = img.shape[1]
    img_width = img.shape[2]
    
    # draw bounding boxes
    # bounding box in (xmin, ymin, xmax, ymax) format
    #bbox1 = [30, 45, 330, 450]
    #bbox2 = [320, 150, 440, 460]
    classes, bbox = get_bboex(bb_txt_path, img_height, img_width)
    bbox = torch.tensor(bbox, dtype=torch.int)
    #print(bbox)
    #print(bbox.size())

    # draw bounding boxes on the input image
    img=draw_bounding_boxes(img, bbox, width=3,
    colors=[(255,0,0) if c==0 else (0,255,0) if c==1 else (0, 0, 255) for c in classes])
    img = torchvision.transforms.ToPILImage()(img)
    
    #img = torchvision.transforms.ToPILImage()(img)
    img = np.array(img)
    plt.imshow(img)
    plt.imsave("./{}.pdf".format(file), img)
    # print(classes)

In [18]:
_ = interact(show_images, fdir=directory, file=images)

interactive(children=(Dropdown(description='fdir', options=('29', '15', '13', '19', '14', '11', '82', '22', '2…

# Train YOLOv5

In [19]:
# Cloning yolov5 pytorch implementation
!git clone https://github.com/ultralytics/yolov5 

Cloning into 'yolov5'...
remote: Enumerating objects: 12027, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 12027 (delta 14), reused 6 (delta 4), pack-reused 11996
Receiving objects: 100% (12027/12027), 12.53 MiB | 12.76 MiB/s, done.
Resolving deltas: 100% (8266/8266), done.


In [20]:
# installing requirements.txt
!pip install -r ./yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.7 MB/s 


In [21]:
# Preapre Train and Validation data sequences
train_num = [11, 12, 13, 14, 15, 19, 21, 22, 23, 24, 29, 30, 35, 36, 38, 47]
validation_num = [52, 54, 60, 82]

In [22]:
# Prepare a file which contain train and validation data information and class information
main_train_dir = '/content/VISEM_Tracking_Train_v4/Train'

with open("./yolov5/train_val.yaml", "w") as f:
    
    # Writing train paths to yaml
    f.write("train: [ \n")
    for t in train_num:
        f.write(main_train_dir + f"/{t}," + "\n")
    f.write("]\n\n")
    
    # writing validation paths to yaml
    f.write("val: [\n")
    for v in validation_num:
        f.write(main_train_dir + f"/{v}," + "\n")
    f.write("]\n\n")
    
    # writing number of class parameter
    f.write("nc: 3\n\n")
    
    # Writing class names
    f.write('names: [ "sperm", "cluster", "small_or_pinhead"]')
f.close()

In [23]:
!cat "./yolov5/train_val.yaml"

train: [ 
/content/VISEM_Tracking_Train_v4/Train/11,
/content/VISEM_Tracking_Train_v4/Train/12,
/content/VISEM_Tracking_Train_v4/Train/13,
/content/VISEM_Tracking_Train_v4/Train/14,
/content/VISEM_Tracking_Train_v4/Train/15,
/content/VISEM_Tracking_Train_v4/Train/19,
/content/VISEM_Tracking_Train_v4/Train/21,
/content/VISEM_Tracking_Train_v4/Train/22,
/content/VISEM_Tracking_Train_v4/Train/23,
/content/VISEM_Tracking_Train_v4/Train/24,
/content/VISEM_Tracking_Train_v4/Train/29,
/content/VISEM_Tracking_Train_v4/Train/30,
/content/VISEM_Tracking_Train_v4/Train/35,
/content/VISEM_Tracking_Train_v4/Train/36,
/content/VISEM_Tracking_Train_v4/Train/38,
/content/VISEM_Tracking_Train_v4/Train/47,
]

val: [
/content/VISEM_Tracking_Train_v4/Train/52,
/content/VISEM_Tracking_Train_v4/Train/54,
/content/VISEM_Tracking_Train_v4/Train/60,
/content/VISEM_Tracking_Train_v4/Train/82,
]

nc: 3

names: [ "sperm", "cluster", "small_or_pinhead"]

In [24]:
# Train
!python ./yolov5/train.py --img 640 --batch 16 --epochs 10 --data ./yolov5/train_val.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=./yolov5/train_val.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-53-gf0e5a60 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.01

# Test model 

In [37]:
!python /content/yolov5/detect.py --weights '/content/yolov5/runs/train/exp/weights/best.pt' --source '/content/VISEM_Tracking_Train_v4/Train/60/60.mp4'

Streaming output truncated to the last 5000 lines.
video 17/20 (883/1440) /content/VISEM_Tracking_Train_v4/Train/52/52.mp4: 480x640 12 sperms, 7.4ms
video 17/20 (884/1440) /content/VISEM_Tracking_Train_v4/Train/52/52.mp4: 480x640 12 sperms, 8.2ms
video 17/20 (885/1440) /content/VISEM_Tracking_Train_v4/Train/52/52.mp4: 480x640 12 sperms, 10.5ms
video 17/20 (886/1440) /content/VISEM_Tracking_Train_v4/Train/52/52.mp4: 480x640 13 sperms, 1 small_or_pinhead, 7.3ms
video 17/20 (887/1440) /content/VISEM_Tracking_Train_v4/Train/52/52.mp4: 480x640 12 sperms, 1 small_or_pinhead, 7.5ms
video 17/20 (888/1440) /content/VISEM_Tracking_Train_v4/Train/52/52.mp4: 480x640 12 sperms, 1 small_or_pinhead, 9.2ms
video 17/20 (889/1440) /content/VISEM_Tracking_Train_v4/Train/52/52.mp4: 480x640 12 sperms, 7.5ms
video 17/20 (890/1440) /content/VISEM_Tracking_Train_v4/Train/52/52.mp4: 480x640 12 sperms, 8.0ms
video 17/20 (891/1440) /content/VISEM_Tracking_Train_v4/Train/52/52.mp4: 480x640 12 sperms, 7.7ms
video 

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
!cp -av '/content/VISEM_Tracking_Train_v4' '/content/drive/MyDrive/2022_Selab/MediaEval/code'

Streaming output truncated to the last 5000 lines.
'/content/VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_556_with_ftid.txt' -> '/content/drive/MyDrive/2022_Selab/MediaEval/code/VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_556_with_ftid.txt'
'/content/VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_557_with_ftid.txt' -> '/content/drive/MyDrive/2022_Selab/MediaEval/code/VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_557_with_ftid.txt'
'/content/VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_558_with_ftid.txt' -> '/content/drive/MyDrive/2022_Selab/MediaEval/code/VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_558_with_ftid.txt'
'/content/VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_559_with_ftid.txt' -> '/content/drive/MyDrive/2022_Selab/MediaEval/code/VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_559_with_ftid.txt'
'/content/VISEM_Tracking_Train_v4/Train/60/labels_ftid/60_frame_55_with_ftid.txt' -> '/content/drive/MyDrive/2022

In [52]:
cd '/content/drive/MyDrive/2022_Selab/MediaEval/code'

/content/drive/MyDrive/2022_Selab/MediaEval/code


In [53]:
!python '/content/drive/MyDrive/2022_Selab/MediaEval/code/yolov5/detect.py' --weights '/content/drive/MyDrive/2022_Selab/MediaEval/code/yolov5/runs/train/exp/weights/best.pt' --source '/content/drive/MyDrive/2022_Selab/MediaEval/code/VISEM_Tracking_Train_v4/Train/12/images/12_frame_1003.jpg'

detect: weights=['/content/drive/MyDrive/2022_Selab/MediaEval/code/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/2022_Selab/MediaEval/code/VISEM_Tracking_Train_v4/Train/12/images/12_frame_1003.jpg, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.2-53-gf0e5a60 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/drive/MyDrive/2022_Selab/MediaEval/code/VISEM_Tracking_Train_v4/Train/12/images/12_frame_1003.jpg: 480x640 21 sperms, 1 cluster, 2 small_or_pinheads, 12.4ms
Speed: 0.7ms pre-process, 12